In [ ]:
!pip install beautifulsoup4
!pip install requests 
!pip install requests-html
!pip3 install fake-useragent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 KB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 KB 12.8 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1270 sha256=3e055ca29e00d613fb86877eccc31172a06014d3304894d94a0edaeba87ff64e
  Stored in directory: /root/.cache/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24589 sha256=7a94c983bbaada9b7ebf47588aef7130a31a59d69764d7

In [ ]:
import sys 
import time 
from bs4 import BeautifulSoup 
import requests 
import pandas as pd
import numpy as np
import re
import csv
import urllib.request

from datetime import datetime
from urllib.request import Request, urlopen
from fake_useragent import UserAgent
from google.colab import files


def english_months_converter(malay_month):
    if "Januari" in malay_month:
        malay_month = malay_month.replace('Januari', 'January')
    elif 'Februari' in malay_month:
        malay_month = malay_month.replace('Februari', 'February')
    elif 'Mac' in malay_month:
        malay_month = malay_month.replace('Mac', 'March')
    elif 'Mei' in malay_month:
        malay_month = malay_month.replace('Mei', 'May')
    elif 'Jun' in malay_month:
        malay_month = malay_month.replace('Jun', 'June')
    elif 'Julai' in malay_month:
        malay_month = malay_month.replace('Julai', 'July')
    elif 'Ogos' in malay_month:
        malay_month = malay_month.replace('Ogos', 'August')
    elif 'Oktober' in malay_month:
        malay_month = malay_month.replace('Oktober', 'October')
    elif 'Disember' in malay_month:
        malay_month = malay_month.replace('Disember', 'December')    
    return malay_month

# url to scrape 
url = 'https://www.kosmo.com.my/?s=vaksin'
pages_to_get = 5

with open('kosmo.csv', 'w', newline='') as f:
  writer = csv.writer(f)
  headers = ["Article Title", "Author", "Date & Time", "Link", "Content"]
  writer.writerow(headers)
  
  for page in range(1, pages_to_get + 1):
        print('Processing Page: ', page)
        if page == 1:
            url = 'https://www.kosmo.com.my/?s=vaksin'
        else:
            url = 'https://www.kosmo.com.my/page/'+ str(page) + '/?s=vaksin'

        try:
      # response is equivalent to enter a key in chrome 

            response = requests.get(url, headers={'User-Agent': UserAgent().random})

      # this link give valid status code: 200 --> pass 
      # print(page.status_code)

        except Exception as e:
            error_type, error_obj, error_info = sys.exc_info()
            print('Error Link: ', url)
            print(error_type, 'Line: ', error_info.tb_lineno)
      # ignore this paage and move on to next one
            continue 

    # delay by 2 seconds
        time.sleep(2)

        soup = BeautifulSoup(response.text, 'html.parser')
    # inspect element attribute type and its names to take their information

        attrs_code = 'elementor-post__text'
        links = soup.find_all('div', attrs={'class':attrs_code})
    # print(len(links))
    # print(links[0])
    
        div = soup.find('div', attrs={'class':attrs_code})

        for link in links:
      # Within div tag, find h3 tag, then within h3 find 'a' tag, extract its text
            Article_Title = link.find('h3').find('a').text

      # Ex: Date and Time
            Date_Time = english_months_converter(link.find('div', attrs={'elementor-post__meta-data'}).text)

      # Extract value of href attribute from 'a' tag using dictionary-style access

      # Print the data to the screen
            Link = link.find('h3').find('a')['href']
            # opens link for each article and extracts html
            pages = Request(Link, headers={'User-Agent': UserAgent().random}) 
            infile = urlopen(pages).read()
            article_page = BeautifulSoup(infile, 'html.parser')
            # obtains a list of the span class and the 5th item gives author name
            author_text = article_page.find_all('span', attrs={'class': 'elementor-icon-list-text'})[4].get_text()
            Author = author_text[5:]
            # content for each article
            Content = ''
            for content in article_page.find_all('p'):
                Content += content.get_text()
            Content = Content[28:].encode("utf-8")
            writer.writerow([Article_Title, Author, Date_Time, Link, Content])

        print('CSV file saved successfully for Page: ' + str(page))
files.download('kosmo.csv')
print('CSV file complete')

Processing Page:  1
CSV file saved successfully for Page: 1
Processing Page:  2
CSV file saved successfully for Page: 2
Processing Page:  3
CSV file saved successfully for Page: 3
Processing Page:  4
CSV file saved successfully for Page: 4
Processing Page:  5
CSV file saved successfully for Page: 5


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

CSV file complete


In [ ]:
pd = pd.read_csv("kosmo.csv")

display(pd)

,Article Title,Author,Date & Time,Link,Content
0,"\n24 negara catat kes taun, WHO tingkatkan ama...",KOSMO,"\n\n24 March 2023, 8:25 am \n",https://www.kosmo.com.my/2023/03/24/24-negara-...,"b'\xc2\xbb 24 negara catat kes taun, WHO tingk..."
1,"\nKerjasama Megaklinik Zahran, Selcare Nursing...",KOSMO!,"\n\n19 March 2023, 1:38 pm \n",https://www.kosmo.com.my/2023/03/19/kerjasama-...,"b'\xc2\xbb Kerjasama Megaklinik Zahran, Selcar..."
2,\nDewan Rakyat: Trend jangkitan Covid-19 dalam...,MOHD RASDAN AHMAD,"\n\n8 March 2023, 2:32 pm \n",https://www.kosmo.com.my/2023/03/08/dewan-raky...,b'\xc2\xbb Dewan Rakyat: Trend jangkitan Covid...
3,\nChina tewaskan Barat dalam ‘perlumbaan’ tekn...,KOSMO!,"\n\n4 March 2023, 8:17 am \n",https://www.kosmo.com.my/2023/03/04/china-tewa...,b'\xc2\xbb China tewaskan Barat dalam \xe2\x80...
4,\nDewan Rakyat: Ranking bola sepak Malaysia an...,ZULKIFLI MANZOR,"\n\n2 March 2023, 10:04 am \n",https://www.kosmo.com.my/2023/03/02/dewan-raky...,b'\xc2\xbb Dewan Rakyat: Ranking bola sepak Ma...
5,\nWHO bimbang penularan selesema burung,KOSMO!,"\n\n26 February 2023, 8:15 am \n",https://www.kosmo.com.my/2023/02/26/who-bimban...,b'\xc2\xbb WHO bimbang penularan selesema buru...
6,"\nDapatkan rawatan jika golongan komorbid, war...",MUHAMMAD FARID AHMAD TARMIJI,"\n\n20 February 2023, 9:00 am \n",https://www.kosmo.com.my/2023/02/20/dapatkan-r...,b'\xc2\xbb Dapatkan rawatan jika golongan komo...
7,"\nDoktor, petugas kesihatan deklamasi puisi",MUHAMMAD FARID AHMAD TARMIJI,"\n\n17 February 2023, 8:10 am \n",https://www.kosmo.com.my/2023/02/17/doktor-pet...,"b'\xc2\xbb Doktor, petugas kesihatan deklamasi..."
8,"\n93 ventilator tidak dapat digunakan, kerajaa...",MOHD. HUSNI MOHD. NOOR,"\n\n16 February 2023, 4:05 pm \n",https://www.kosmo.com.my/2023/02/16/93-ventila...,b'\xc2\xbb 93 ventilator tidak dapat digunakan...
9,\nLaporan Audit: MySejahtera ‘diserang’ 1.12 j...,MOHD. HUSNI MOHD. NOOR,"\n\n16 February 2023, 2:50 pm \n",https://www.kosmo.com.my/2023/02/16/mysejahter...,b'\xc2\xbb Laporan Audit: MySejahtera \xe2\x80...
